# Módulos útiles para ficheros grib

Para abrir el notebook en Colab pulsa el botón:

<a href="https://colab.research.google.com/github/jlcasador/curso_formatos_meteo/blob/main/Modulos_para_grib.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[GRIB](https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them) es un acrónimo cuyas siglas corresponden a "General Regularly distributed Information in Binary form".  
Es un formato estándar de la Organización Meteorológica Mundial (OMM) para archivar e intercambiar datos meteorológicos.

Esta foto puede representar la jerarquía de paquetes que vamos a usar en este curso:

![jerarquía de módulos](matroshka_grib_60.jpg)

### Instalación

#### Instalación de eccodes utilizando CMake

In [ ]:
!wget https://confluence.ecmwf.int/download/attachments/45757960/eccodes-2.30.0-Source.tar.gz?api=v2 -O eccodes-2.30.0-Source.tar.gz
!tar -xzf  /content/eccodes-2.30.0-Source.tar.gz
!mkdir build ; cd build
%cd build
!cmake -DCMAKE_INSTALL_PREFIX=eccodes ../eccodes-2.30.0-Source
!make -j 4
!make install
%cd /content

In [4]:
import os
PATH = os.getenv('PATH')
os.environ['PATH'] = ':'.join([PATH, '/content/build/eccodes/bin'])

In [ ]:
!wget https://github.com/waveology/tests/raw/main/local_grib_definitions.rar
!wget https://github.com/waveology/tests/raw/main/data.grib
!unrar x -o+ /content/local_grib_definitions.rar

In [7]:
!codes_info


ecCodes Version 2.28.0

Definition files path from environment variable ECCODES_DEFINITION_PATH=/usr/local/apps/gcc/11.2.0/eccodes/2.28.0/share/eccodes/definitions:/usr/local/apps/global/harmonie_definitions/1.1/grib/definitions
Full definition files path=/usr/local/apps/gcc/11.2.0/eccodes/2.28.0/share/eccodes/definitions:/usr/local/apps/global/harmonie_definitions/1.1/grib/definitions

Default SAMPLES path is used: /usr/local/apps/gcc/11.2.0/eccodes/2.28.0/share/eccodes/samples
SAMPLES path can be changed by setting the ECCODES_SAMPLES_PATH environment variable.



In [ ]:
os.environ['GRIB_DEFINITION_PATH'] = '/content/local_grib_definitions:/usr/local/share/grib_api/definitions'

#### Instalación de eccodes con Conda

In [ ]:
!wget https://github.com/waveology/tests/raw/main/local_grib_definitions.rar
!wget https://github.com/waveology/tests/raw/main/data.grib
!unrar x -o+ /content/local_grib_definitions.rar && rm -rf /content/local_grib_definitions.rar

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

In [ ]:
!conda install -q eccodes -c conda-forge

In [ ]:
!codes_info

In [ ]:
import os
os.environ['GRIB_DEFINITION_PATH'] = '/content/local_grib_definitions:/usr/local/share/grib_api/definitions'
!echo $GRIB_DEFINITION_PATH

#### Instalación de otros paquetes

In [ ]:
#!pip install xarray # xarray está instalado por defecto
!pip install cartopy
# metview
#!pip install climetlab

### Grib_tools

Las [grib tools](https://confluence.ecmwf.int/display/ECC/GRIB+tools) son un conjunto de comandos que permiten 

In [ ]:
!grib_ls ecmwf_surface.grib

In [ ]:
# Cuidado con mayúsculas y espacios!
!grib_ls -p paramId,shortName,dataDate,dataTime,step ecmwf_surface.grib

In [ ]:
!grib_dump ecmwf_surface.grib | head -n 120

### Módulo eccodes

https://sites.ecmwf.int/docs/eccodes/namespaceec_codes.html

In [ ]:
from eccodes import *

In [ ]:
f = open("ecmwf_surface.grib", "rb")

In [ ]:
gid = codes_new_from_file(f, CODES_PRODUCT_GRIB)

In [ ]:
codes_get(gid, "paramId")

In [ ]:
codes_release(gid)

In [ ]:
f.close()

Efecto de la interfaz de python:

![flujo basico C](flujo_basico_C.png)

![flujo_basico_python](flujo_basico_python.png)

Algo un poco más complicado:

In [ ]:
def nearest_value(filename, input_paramid, latitude, longitude):
    
    f = open(filename, "rb")
    values = {}
    
    while True:
        gid = codes_new_from_file(f, CODES_PRODUCT_GRIB)
        if gid is None: break
        
        paramid = codes_get(gid, "paramId")
        step = codes_get(gid, "step")
        if paramid==input_paramid:
            values[step] = codes_grib_find_nearest(gid, latitude, longitude)[0].value
        
        codes_release(gid)

    f.close()
    
    return values

In [ ]:
nearest_value("ecmwf_surface.grib", 167, 40, -5)

https://www.ecmwf.int/en/newsletter/169/computing/advanced-regridding-metview

https://metview.readthedocs.io/en/latest/

### Módulo xarray

In [ ]:
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

In [ ]:
ds = xr.load_dataset("ecmwf_surface.grib", engine="cfgrib")

In [ ]:
t_h12 = ds.sel(step = timedelta(hours=12)).t2m - 273.15

In [ ]:
t_h12.plot()

In [ ]:
plt.figure(figsize=(8,4.5))
ax = plt.axes(projection=ccrs.Mercator())
ax.coastlines()
t_h12.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels= np.arange(-20,25,1))
plt.show()

### Módulo climetlab